# 1. Install and Import Dependencies

In [1]:
%pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 1982.2 MB 1.4 kB/s  eta 0:00:01   |▊                               | 48.1 MB 7.7 MB/s eta 0:04:13     |█▌                              | 93.9 MB 22.5 MB/s eta 0:01:24     |██▋                             | 163.6 MB 26.9 MB/s eta 0:01:08     |██▉                             | 173.7 MB 26.8 MB/s eta 0:01:08     |███▋                            | 223.4 MB 25.0 MB/s eta 0:01:11     |████                            | 251.9 MB 14.9 MB/s eta 0:01:56     |████▉                           | 300.0 MB 26.3 MB/s eta 0:01:04     |█████                           | 305.7 MB 26.3 MB/s eta 0:01:04     |█████▌                          | 343.9 MB 23.1 MB/s eta 0:01:11     |█████▊                          | 351.5 MB 23.1 MB/s eta 0:01:11     |██████                          | 376.4 MB 4.2 MB/s eta 0:06:21     |██████                          | 377.7 MB 4.2 MB/s eta 0:06:21     |██████▏              

In [2]:
%pip install transformers requests beautifulsoup4 pandas numpy

     |████████████████████████████████| 7.2 MB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 12.4 MB 27.4 MB/s eta 0:00:01
     |████████████████████████████████| 341 kB 27.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 26.1 MB/s eta 0:00:01
     |████████████████████████████████| 236 kB 15.3 MB/s eta 0:00:01
     |████████████████████████████████| 769 kB 26.2 MB/s eta 0:00:01
     |████████████████████████████████| 7.8 MB 27.6 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 4.8 MB/s  eta 0:00:01
     |████████████████████████████████| 163 kB 25.0 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart th

In [4]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate Model

In [19]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 3. Encode and Calculate Sentiment

In [28]:
tokens = tokenizer.encode('Horrible experience would never come back again ever', return_tensors='pt')

In [29]:
result = model(tokens)

In [30]:
result.logits

tensor([[ 3.9117,  1.7635, -0.3847, -2.4659, -2.1921]],
       grad_fn=<AddmmBackward>)

In [31]:
int(torch.argmax(result.logits))+1

1

# 4. Collect Reviews

In [42]:
r = requests.get("http://www.crossculturerestaurant.com/testimonials.aspx")
soup = BeautifulSoup(r.text, 'html.parser')
#regex = re.compile('.*comment.*')
results = soup.find_all('div', class_="TestimonialText")


In [46]:
reviews = [result.text.strip() for result in results]


In [47]:
reviews


['I love this northern Indian cuisine. The potato flat bread was spectacular, even turning one of my non- believer friends to the Indian cuisine light side. The samosas are beyond belief delicious, and the sauces that go with them make them perfect. Very good.',
 'Lively atmosphere with nice Indian music playing on a decent sound system.  Nice staff and good service.  Food was good and freshly prepared.  Samosas had way too much chili flakes.  I like them with spice but not loads of chili flakes.  Tandori chicken was tasty though hotter in terms of spice than I am used too.  I recommend trying this place especially if you like hot and spicy food.',
 "My new favorite Indian place in the area by a wide margin.  I'm still a fan some dishes elsewhere (best veggie samosas: Mehek, best naan: Masala Grill), but overall, this place blows the other guys away.  All of the chicken entrees I've tried have been outstanding, including tikka masala, korma, and cashmere.  The grilled chicken appetizer

# 5. Load Reviews into DataFrame and Score

In [48]:
import numpy as np
import pandas as pd

In [49]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [50]:
df['review'].iloc[0]

'I love this northern Indian cuisine. The potato flat bread was spectacular, even turning one of my non- believer friends to the Indian cuisine light side. The samosas are beyond belief delicious, and the sauces that go with them make them perfect. Very good.'

In [51]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [52]:
sentiment_score(df['review'].iloc[1])

4

In [53]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [54]:
df

,review,sentiment
0,I love this northern Indian cuisine. The potat...,5
1,Lively atmosphere with nice Indian music playi...,4
2,My new favorite Indian place in the area by a ...,5
3,We went to Cross Culture Princeton for dinner ...,5
4,My family and I have been to Cross Culture twi...,4
5,Fantastic food! It was very fresh and flavorfu...,5
6,"Awesome food , best service and beautiful plac...",5
7,Best indian food I have had in my life. Great ...,5
8,"Great food, great ambiance, poor service. We w...",1
9,Pleasantly surprised. Loved it the first time ...,5


In [55]:
df['review'].iloc[3]

'We went to Cross Culture Princeton for dinner and the food was amazingly tasty, I have tried other Indian restaurants in the area but there is no comparison in the taste. The service was outstanding in warm cozy and romantic atmosphere with classical music in the background. We had sizzling Chicken and Lamb kebab for appetizer. We enjoyed the vegetarian and non-vegetarian entree with fresh Tandoori Naan bread and basmati rice. We highly recommend this place to everyone.'